## README

make input files and command file(bash) for running VESPA

Input Files
Required: A single FASTA file containing all your wildtype sequences (note: this file can contain any number of sequences).

Optional: If you are only interested in a subset of possible mutations (specific mutations), you can add -m mutations.txt to the code line in Quickstart (note: per default all mutations are considered). Click here to head to the file format explanation of mutations.txt.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import requests
import json


In [2]:
# read in all gene names
gene_list_raw =pd.read_csv("230323_EST_ENSG_GENE_new.csv")
uni_list = gene_list_raw['Uniprot'].tolist()
uni_list

['O95136',
 'P59533',
 'P59538',
 'P59541',
 'P59543',
 'P49019',
 'P59537',
 'Q96G91',
 'Q96RI9',
 'P59534',
 'P59540',
 'P59535',
 'P59536',
 'Q49SQ1',
 'Q15722',
 'Q99500',
 'Q9NYV9',
 'Q9NYV8',
 'P59542',
 'Q9BXB1',
 'Q99678',
 'Q9UBS5',
 'P50391',
 'Q9Y2T5',
 'Q9BXA5',
 'P47901',
 'P21730',
 'Q9BXC0',
 'P35367',
 'P34972',
 'P13945',
 'O00144',
 'Q99679',
 'P25089',
 'Q7Z602',
 'P51582',
 'Q5NUL3',
 'Q7RTR8',
 'P59551',
 'O14843',
 'Q01718',
 'P49146',
 'P08912',
 'P21728',
 'Q9H1C0',
 'P46092',
 'Q9NS66',
 'O43194',
 'P51679',
 'P48145',
 'Q96LB1',
 'P46091',
 'Q15760',
 'Q9Y5Y4',
 'Q8TDS4',
 'O43603',
 'Q9NSD7',
 'Q86SM5',
 'Q86VZ1',
 'P46089',
 'Q9GZN0',
 'Q9BPV8',
 'Q8IZ08',
 'Q9UKP6',
 'P08172',
 'Q9BZJ7',
 'P30559',
 'P25025',
 'P50052',
 'Q5UAW9',
 'P08173',
 'P30874',
 'Q14332',
 'Q6DWJ6',
 'Q7Z7M1',
 'Q8TDU6',
 'Q96LB0',
 'Q96LA9',
 'P28221',
 'P30939',
 'Q8TE23',
 'P08908',
 'Q8NGU9',
 'P46093',
 'Q9H461',
 'P33032',
 'Q8NFN8',
 'P41231',
 'Q8TDT2',
 'Q86SP6',
 'O14626',

In [11]:
gpcr_valid_missense = pd.read_csv('gpcr_valid_missense.csv')
gpcr_valid_missense

,vep_input,mutation_pos,SYMBOL,Amino_acids,protein,SIFT,PolyPhen,BayesDel_addAF_score,BayesDel_noAF_score,CADD_phred,...,MutationAssessor_score,PROVEAN_score,PrimateAI_score,REVEL_score,VEST4_score,GERP++_RS,SiPhy_29way_logOdds,MutPred_score,mutation_from,mutation_to
0,ENST00000294954:p.Ala572Val,572,LHCGR,A/V,lshr_human,deleterious(0),probably_damaging(0.998),0.271259,0.151868,28.4,...,"2.685,.","-3.47,-3.51",0.647560358047,"0.613,0.613","0.778,0.789",5.68,18.7926,0.935,A,V
1,ENST00000294954:p.Ala593Pro,593,LHCGR,A/P,lshr_human,deleterious(0),probably_damaging(1),0.263129,0.14019,25.7,...,"3.28,.","-3.34,-3.56",0.664010286331,"0.595,0.595","0.897,0.938",5.68,18.7926,0.886,A,P
2,ENST00000294954:p.Asp564Gly,564,LHCGR,D/G,lshr_human,deleterious(0),probably_damaging(0.999),0.302076,0.196135,28.5,...,"3.49,.","-6.01,-5.99",0.665593266487,"0.842,0.842","0.923,0.954",5.68,15.1242,0.897,D,G
3,ENST00000294954:p.Asp578Gly,578,LHCGR,D/G,lshr_human,deleterious(0),possibly_damaging(0.685),0.294007,0.184544,27.4,...,"3.715,.","-5.96,-6.04",0.658861994743,"0.756,0.756","0.84,0.927",5.68,15.1242,0.936,D,G
4,ENST00000294954:p.Asp578His,578,LHCGR,D/H,lshr_human,deleterious(0),probably_damaging(0.952),0.275656,0.158184,24.6,...,"3.715,.","-5.94,-6.02",0.713786900043,"0.500,0.500","0.828,0.864",4.8,14.1045,0.877,D,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,ENST00000276198:p.Cys360Ala,360,HTR2C,C/A,5ht2c_human,deleterious(0.02),benign(0.015),-,-,-,...,-,-,-,-,-,-,-,-,C,A
2481,ENST00000276198:p.Ser361Ala,361,HTR2C,S/A,5ht2c_human,deleterious(0),probably_damaging(0.998),0.270807,0.151219,24.3,...,".,.","-2.26,-2.26",0.640748023987,"0.821,0.821","0.623,0.65",5.13,11.8568,0.886,S,A
2482,ENST00000276198:p.Tyr368Ala,368,HTR2C,Y/A,5ht2c_human,deleterious(0),probably_damaging(1),-,-,-,...,-,-,-,-,-,-,-,-,Y,A
2483,ENST00000276198:p.Tyr368Cys,368,HTR2C,Y/C,5ht2c_human,deleterious(0),probably_damaging(1),0.622314,0.656134,24.9,...,".,.","-7.09,-7.09",0.834922552109,"0.881,0.881","0.964,0.961",5.13,11.8568,0.867,Y,C


## Fetch FASTA file individually into a folder

In [3]:
def get_fasta(uniprot_id):
    # uniprot_id = "P07550"
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"

    response = requests.get(url)
    if response.ok:
        fasta_data = response.text
        with open(f"fasta/{uniprot_id}.fasta", "w") as f:
            f.write(fasta_data)
    else:
        print(f"Error retrieving FASTA sequence: {response.status_code} {response.reason}")

In [6]:
#fetch the sequence file
for i in range(len(uni_list)):
    get_fasta(uni_list[i])

## Fetch FASTA file altogether into a file

In [4]:
def get_fasta_merge(uniprot_id):
    # uniprot_id = "P07550"
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"

    response = requests.get(url)
    if response.ok:
        fasta_data = response.text
        with open("sequence.fasta", "a") as f:
            f.write(fasta_data)
    else:
        print(f"Error retrieving FASTA sequence: {response.status_code} {response.reason}")

In [7]:
# Load the array from the text file
unq_uniid_list = np.unique(gpcr_valid_missense['protein'].tolist())
unq_uniid_list

array(['5ht1a_human', '5ht1b_human', '5ht1d_human', '5ht2a_human',
       '5ht2b_human', '5ht2c_human', '5ht4r_human', '5ht5a_human',
       '5ht6r_human', '5ht7r_human', 'aa1r_human', 'aa2ar_human',
       'aa2br_human', 'aa3r_human', 'acm1_human', 'acm2_human',
       'acm3_human', 'acm4_human', 'acm5_human', 'ada1a_human',
       'ada1b_human', 'ada2b_human', 'adrb1_human', 'adrb2_human',
       'adrb3_human', 'calrl_human', 'casr_human', 'ccr1_human',
       'ccr2_human', 'ccr3_human', 'ccr5_human', 'ccr8_human',
       'cnr2_human', 'crfr1_human', 'cxcr1_human', 'cxcr2_human',
       'cxcr3_human', 'cxcr4_human', 'drd1_human', 'drd2_human',
       'drd3_human', 'drd5_human', 'ednrb_human', 'fshr_human',
       'ghrhr_human', 'gipr_human', 'glp1r_human', 'glr_human',
       'grm1_human', 'grm2_human', 'grm5_human', 'hrh1_human',
       'hrh2_human', 'hrh3_human', 'hrh4_human', 'lshr_human',
       'mc4r_human', 'p2ry2_human', 'p2y12_human', 'pth1r_human',
       'pth2r_human', 'sct

In [8]:
# put all fasta sequence into one fasta file
for i in range(len(unq_uniid_list)):
    get_fasta_merge(unq_uniid_list[i])
    

## Make mutation list for all target proteins into a txt file

VESPA! mutation position = position - 1!!!

In [12]:

for i in range(gpcr_valid_missense.shape[0]):
    # get the uniprot id according to entryname and convert the Series to a string with index excluded
    uniprot_id = gene_list_raw.loc[gene_list_raw['protein']==gpcr_valid_missense.iloc[i,4],'Uniprot'].to_string(index=False)
    position = gpcr_valid_missense.iloc[i,1]-1
    mut_info = str(gpcr_valid_missense.iloc[i,30])+str(position)+str(gpcr_valid_missense.iloc[i,31])
    with open("mutations.txt", 'a') as file:
        # write a new line to the file
        file.write(uniprot_id+"_"+mut_info+"\n")
    

fail to run vespa (which provides conservation scores and log-odds ratio) because the out of memory
only run VESPAI (conservation scores)

after getting the results, run the below parts:

## Process the output of VESPAI and output as csv file

In [14]:
import json
import pandas as pd
import os

# read in the map.json file
with open('output/map.json', 'r') as f:
    id_map = json.load(f)

# create an empty dataframe to store the combined data
combined_df = pd.DataFrame(columns=['Uniprot_ID', 'protein', 'Mutant', 'VESPAl'])

# loop over each file in the output folder
for filename in os.listdir('output'):
    if filename.endswith('.csv'):
        # get the uniprot id for this file from the filename
        id_num = int(filename.split('.')[0])
        uniprot_id = id_map[str(id_num)]

        protein_name = gene_list_raw.loc[gene_list_raw['Uniprot']== uniprot_id,'protein'].to_string(index=False)
        
        # read in the csv file and split the information on ';'
        csv_data = pd.read_csv(os.path.join('output', filename), delimiter=';')
        
        # add the uniprot id column and concatenate with the combined dataframe
        csv_data['Uniprot_ID'] = uniprot_id
        csv_data['protein'] = protein_name
        combined_df = pd.concat([combined_df, csv_data], ignore_index=True)

# print the resulting dataframe
print(combined_df)


     Uniprot_ID      protein Mutant    VESPAl
0        Q13639  5ht4r_human  Y301A  0.587059
1        Q13639  5ht4r_human  Y301F  0.366272
2        Q13639  5ht4r_human  N278L  0.592905
3        Q13639  5ht4r_human  F275V  0.554260
4        Q13639  5ht4r_human  F274A  0.581045
...         ...          ...    ...       ...
2480     Q03431  pth1r_human  V238A  0.325055
2481     Q03431  pth1r_human  K239A  0.506099
2482     P41587  vipr2_human  Y183F  0.166512
2483     P41587  vipr2_human  K178Q  0.401903
2484     P41587  vipr2_human  R171Q  0.552963

[2485 rows x 4 columns]


In [15]:
# extract amino acid, position, and end amino acid from 'Mutant' column
combined_df[['mutation_from', 'mutation_pos', 'mutation_to']] = combined_df['Mutant'].str.extract(r'(\D)(\d+)(\D)')

# convert 'Position' column to integer type and plus 1 for further merge with GPCRdb data
combined_df['mutation_pos'] = combined_df['mutation_pos'].astype(int)+1
combined_df = combined_df.drop('Mutant', axis=1)
combined_df

,Uniprot_ID,protein,VESPAl,mutation_from,mutation_pos,mutation_to
0,Q13639,5ht4r_human,0.587059,Y,302,A
1,Q13639,5ht4r_human,0.366272,Y,302,F
2,Q13639,5ht4r_human,0.592905,N,279,L
3,Q13639,5ht4r_human,0.554260,F,276,V
4,Q13639,5ht4r_human,0.581045,F,275,A
...,...,...,...,...,...,...
2480,Q03431,pth1r_human,0.325055,V,239,A
2481,Q03431,pth1r_human,0.506099,K,240,A
2482,P41587,vipr2_human,0.166512,Y,184,F
2483,P41587,vipr2_human,0.401903,K,179,Q


In [16]:
combined_df.to_csv("gpcr_vespa_df.csv",index=False)